In [1]:
// hacky way to set java path since macos breaks this
val jniPath = "/Users/shawnmeier/software/z3/build"
val newPath = Array(jniPath) ++  System.getProperty("java.library.path").split(":")
System.setProperty("java.library.path",newPath.distinct.mkString(":"))
//set sys_paths to null so that java.library.path will be reevaluated next time it is needed
val sysPathsField = classOf[ClassLoader].getDeclaredField("sys_paths");
sysPathsField.setAccessible(true);
sysPathsField.set(null, null);

val path = "/Users/shawnmeier/Documents/source/bounder/soot_hopper/target/scala-2.13/soot_hopper-assembly-0.1.jar"
//unzip -p jdiff.jar META-INF/MANIFEST.MF
interp.load.cp(os.Path(path))

import $ivy.`org.plotly-scala:plotly-almond_2.13:0.8.2`
import $ivy.`com.github.pathikrit::better-files:3.9.1`

import $ivy.`com.lihaoyi:ujson_2.13:1.3.8`
// import $ivy.`edu.colorado.plv.bounder:soot_hopper_2.13:0.1`

import plotly._, element._, layout._, Plotly._
import ujson.Value
import sys.process._


jniPath: String = "/Users/shawnmeier/software/z3/build"
newPath: Array[String] = Array(
  "/Users/shawnmeier/software/z3/build",
  "/Users/shawnmeier/Library/Java/Extensions",
  "/Library/Java/Extensions",
  "/Network/Library/Java/Extensions",
  "/System/Library/Java/Extensions",
  "/usr/lib/java",
  "."
)
res0_2: String = "/Users/shawnmeier/Library/Java/Extensions:/Library/Java/Extensions:/Network/Library/Java/Extensions:/System/Library/Java/Extensions:/usr/lib/java:."
sysPathsField: java.lang.reflect.Field = private static java.lang.String[] java.lang.ClassLoader.sys_paths
path: String = "/Users/shawnmeier/Documents/source/bounder/soot_hopper/target/scala-2.13/soot_hopper-assembly-0.1.jar"
import $ivy.$                                          

import $ivy.$                                         


import $ivy.$                             
// import $ivy.`edu.colorado.plv.bounder:soot_hopper_2.13:0.1`


import plotly._, element._, layout._, Plotly._

import ujson.Value

import sy

In [2]:
import better.files._

import scala.util.Random
import edu.colorado.plv.bounder.Driver
import edu.colorado.plv.bounder.Action
import edu.colorado.plv.bounder.RunConfig
import upickle.default.read
import upickle.default.write
import scala.collection.parallel.CollectionConverters.{ImmutableSetIsParallelizable, IterableIsParallelizable}
import edu.colorado.plv.bounder.ExperimentsDb
import edu.colorado.plv.bounder.BounderUtil
import scala.concurrent.duration._
import scala.language.postfixOps
import slick.driver.H2Driver.api._
import slick.jdbc.GetResult
import slick.jdbc.SQLActionBuilder
import scala.concurrent.Await
Driver.setZ3Path("/Users/shawnmeier/software/z3/build")
BounderUtil.setEnv(Map("DYLD_LIBRARY_PATH" -> "/Users/shawnmeier/software/z3/build","ANDROID_HOME" -> "/Users/shawnmeier/Library/Android/sdk","HOME" -> "/Users/shawnmeier"))

java.library.path set to: /Users/shawnmeier/software/z3/build:/Users/shawnmeier/Library/Java/Extensions:/Library/Java/Extensions:/Network/Library/Java/Extensions:/System/Library/Java/Extensions:/usr/lib/java:.


import better.files._


import scala.util.Random

import edu.colorado.plv.bounder.Driver

import edu.colorado.plv.bounder.Action

import edu.colorado.plv.bounder.RunConfig

import upickle.default.read

import upickle.default.write

import scala.collection.parallel.CollectionConverters.{ImmutableSetIsParallelizable, IterableIsParallelizable}

import edu.colorado.plv.bounder.ExperimentsDb

import edu.colorado.plv.bounder.BounderUtil

import scala.concurrent.duration._

import scala.language.postfixOps

import slick.driver.H2Driver.api._

import slick.jdbc.GetResult

import slick.jdbc.SQLActionBuilder

import scala.concurrent.Await


In [3]:
System.getProperty("user.home")

res2: String = "/Users/shawnmeier"

In [4]:
System.setProperty("user.dir", s"${System.getProperty("user.home")}/Documents/source/bounder/notebooks/ossExp/data");
val expDir = File(System.getProperty("user.dir"))
val baseDir = File(System.getProperty("user.home")) / "Documents/data/ossExperiment"

res3_0: String = "/Users/shawnmeier/Documents/source/bounder/notebooks/ossExp"
expDir: File = /Users/shawnmeier/Documents/source/bounder/notebooks/ossExp/data
baseDir: File = /Users/shawnmeier/Documents/data/ossExperiment

In [ ]:
// Create json files for config runs
def createConfigsFromList():Unit = {
    val mode = Driver.SampleDeref
    val baseConfig = RunConfig(limit = -1, samples = 50)
    case class ApkSample(apk:String, pkg:String, outDir:File)
    val apkSamples = (expDir / "benchmark-list.txt")
        .contentAsString
        .split("\n")map{v => 
            val pkg = v.split(":")(1)
            println(s"package: $pkg")
            val apk = v.split(":")(0)
            val apkDirCfg = apk.toString.replace(baseDir.toString,"${baseDir}")
            val outDirCfg = "${baseDirOut}/" + pkg
            val outDir = expDir / pkg
//             val getSample = outDir / "getSample.json"
//             if(getSample.exists())getSample.delete()
            // val samples = if(pkg.contains("lyonbros")) 90 else 200
            val cfg = baseConfig.copy(apkPath=apkDirCfg, outFolder=Some(outDirCfg), samples=90) // 5 dereference samples per class
//             getSample.append(write[RunConfig](cfg))
            outDir.createIfNotExists(true)
            // sample dereference locations
            Action(Driver.SampleDeref, baseDirOut=Some(expDir.toString), baseDirApk = Some(baseDir.toString), // b u options
                  cfg, filter=Some(pkg))
        }
    apkSamples.foreach{action => 
        println(s"action: ${action}")
        Driver.runAction(action)
        val f = File(s"data/${action.filter.get}.cfg")
        f.overwrite(write[RunConfig](action.config))
    }
}
createConfigsFromList()

package: de.danoeh.antennapod
package: org.connectbot
package: name.boyle.chris
action: Action(SampleDeref,Some(/Users/shawnmeier/Documents/source/bounder/notebooks/ossExp/data),Some(/Users/shawnmeier/Documents/data/ossExperiment),RunConfig(${baseDir}/Antennapod/app/build/outputs/apk/free/debug/app-free-debug.apk,Some(${baseDirOut}/de.danoeh.antennapod),None,TopSpecSet,List(),-1,90,,600,true,None),Some(de.danoeh.antennapod),None,MEM)
java.library.path set to: /Users/shawnmeier/software/z3/build:/Users/shawnmeier/Library/Java/Extensions:/Library/Java/Extensions:/Network/Library/Java/Extensions:/System/Library/Java/Extensions:/usr/lib/java:.
